# 🌱 Gerador de Dataset - Pastagens Brasileiras

Este notebook implementa a geração completa de datasets sintéticos de pastagens brasileiras.

**Características:**
- ✅ Geração automática de milhares de imagens
- ✅ Controle de qualidade integrado
- ✅ Formatação automática para YOLO
- ✅ Distribuição balanceada por biomas/estações
- ✅ Otimizado para Google Colab

In [ ]:
import sys
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Setup projeto
if '/content/img-sinth' not in sys.path:
    sys.path.append('/content/img-sinth')
if not Path('src').exists():
    os.chdir('/content/img-sinth')

# Importações
import torch
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import asdict
import ipywidgets as widgets
from IPython.display import display, clear_output
import time

from src.dataset.generator import DatasetGenerator, GenerationConfig
from src.dataset.yolo_formatter import YOLOFormatter
from src.diffusion.pipeline_manager import PipelineManager
from src.diffusion.prompt_engine import PromptEngine

print("✅ Setup concluído!")
print(f"🖥️ Device: {'CUDA' if torch.cuda.is_available() else 'CPU'}")

# Adicionar projeto ao path
if '/content/img-sinth' not in sys.path:
    sys.path.append('/content/img-sinth')
    
if not Path('src').exists():
    os.chdir('/content/img-sinth')

In [ ]:
# Widgets de configuração
num_images_widget = widgets.IntSlider(
    value=1000,
    min=100,
    max=10000,
    step=100,
    description='Total de Imagens:'
)

resolution_widget = widgets.Dropdown(
    options=[("1024x1024 (Recomendado)", (1024, 1024)), 
             ("768x768 (Rápido)", (768, 768)),
             ("512x512 (Teste)", (512, 512))],
    value=(1024, 1024),
    description='Resolução:'
)

quality_threshold_widget = widgets.FloatSlider(
    value=0.7,
    min=0.4,
    max=0.9,
    step=0.05,
    description='Threshold Qualidade:'
)

use_controlnet_widget = widgets.Checkbox(
    value=True,
    description='Usar ControlNet'
)

# Distribuições
cerrado_pct_widget = widgets.IntSlider(value=50, min=10, max=70, description='Cerrado (%)')
mata_pct_widget = widgets.IntSlider(value=30, min=10, max=50, description='Mata Atl. (%)')
pampa_pct_widget = widgets.IntSlider(value=20, min=10, max=40, description='Pampa (%)')

seca_pct_widget = widgets.IntSlider(value=45, min=20, max=60, description='Seca (%)')
chuvas_pct_widget = widgets.IntSlider(value=40, min=20, max=60, description='Chuvas (%)')
transicao_pct_widget = widgets.IntSlider(value=15, min=5, max=30, description='Transição (%)')

# Layout
config_ui = widgets.VBox([
    widgets.HTML("<h3>⚙️ Configuração do Dataset</h3>"),
    num_images_widget,
    resolution_widget,
    quality_threshold_widget,
    use_controlnet_widget,
    widgets.HTML("<h4>📊 Distribuição por Bioma</h4>"),
    cerrado_pct_widget,
    mata_pct_widget, 
    pampa_pct_widget,
    widgets.HTML("<h4>🌦️ Distribuição Sazonal</h4>"),
    seca_pct_widget,
    chuvas_pct_widget,
    transicao_pct_widget
])

display(config_ui)

## 🚀 Geração do Dataset

In [ ]:
def create_generation_config():
    """Cria configuração baseada nos widgets"""
    
    # Normalizar percentuais
    biome_total = cerrado_pct_widget.value + mata_pct_widget.value + pampa_pct_widget.value
    season_total = seca_pct_widget.value + chuvas_pct_widget.value + transicao_pct_widget.value
    
    config = GenerationConfig(
        num_images=num_images_widget.value,
        resolution=resolution_widget.value,
        output_dir="/content/generated_dataset",
        
        # Distribuições normalizadas
        biome_distribution={
            "cerrado": cerrado_pct_widget.value / biome_total,
            "mata_atlantica": mata_pct_widget.value / biome_total, 
            "pampa": pampa_pct_widget.value / biome_total
        },
        
        season_distribution={
            "seca": seca_pct_widget.value / season_total,
            "chuvas": chuvas_pct_widget.value / season_total,
            "transicao": transicao_pct_widget.value / season_total
        },
        
        quality_distribution={"boa": 0.3, "moderada": 0.4, "degradada": 0.3},
        
        # Parâmetros de geração
        guidance_scale_range=(7.0, 10.0),
        num_inference_steps=25,
        use_controlnet=use_controlnet_widget.value,
        quality_threshold=quality_threshold_widget.value,
        max_retries=3
    )
    
    return config

# Inicializar gerador
dataset_generator = DatasetGenerator(
    config_dir="configs"
)

# Interface de geração
start_generation_btn = widgets.Button(
    description='🚀 Iniciar Geração',
    button_style='success',
    layout=widgets.Layout(width='200px')
)

stop_generation_btn = widgets.Button(
    description='⏹️ Parar',
    button_style='danger',
    layout=widgets.Layout(width='100px'),
    disabled=True
)

progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Progresso:',
    bar_style='info',
    layout=widgets.Layout(width='400px')
)

status_text = widgets.HTML(value="<p>Pronto para iniciar geração</p>")
generation_output = widgets.Output()

generation_active = False
generated_dataset_path = None

def start_generation_callback(btn):
    global generation_active, generated_dataset_path
    
    generation_active = True
    start_generation_btn.disabled = True
    stop_generation_btn.disabled = False
    
    config = create_generation_config()
    
    with generation_output:
        clear_output()
        
        try:
            status_text.value = "<p>🔄 Iniciando geração...</p>"
            
            print("🌱 GERAÇÃO DE DATASET INICIADA")
            print("=" * 50)
            print(f"Total de imagens: {config.num_images}")
            print(f"Resolução: {config.resolution[0]}x{config.resolution[1]}")
            print(f"Threshold de qualidade: {config.quality_threshold}")
            print(f"ControlNet: {'Ativado' if config.use_controlnet else 'Desativado'}")
            print("\n📊 Distribuições:")
            print(f"Biomas: {config.biome_distribution}")
            print(f"Estações: {config.season_distribution}")
            print("=" * 50)
            
            # Executar geração
            start_time = time.time()
            
            generated_dataset_path = dataset_generator.generate_dataset(
                config=config,
                save_metadata=True,
                save_intermediate=False
            )
            
            total_time = time.time() - start_time
            
            # Estatísticas finais
            stats = dataset_generator.get_generation_statistics()
            
            print("\n🎉 GERAÇÃO CONCLUÍDA!")
            print("=" * 50)
            print(f"⏱️ Tempo total: {total_time/60:.1f} minutos")
            print(f"✅ Imagens geradas: {stats['successful_generations']}")
            print(f"🎯 Taxa de sucesso: {stats['successful_generations']/stats['total_attempts']*100:.1f}%")
            print(f"📈 Qualidade média: {stats['avg_quality_score']:.3f}")
            print(f"📂 Dataset salvo em: {generated_dataset_path}")
            
            status_text.value = f"<p>✅ Concluído! {stats['successful_generations']} imagens geradas</p>"
            progress_bar.value = 100
            progress_bar.bar_style = 'success'
            
        except Exception as e:
            print(f"❌ Erro na geração: {e}")
            status_text.value = f"<p>❌ Erro: {str(e)[:100]}...</p>"
            progress_bar.bar_style = 'danger'
        
        finally:
            generation_active = False
            start_generation_btn.disabled = False
            stop_generation_btn.disabled = True

def stop_generation_callback(btn):
    global generation_active
    generation_active = False
    status_text.value = "<p>⏹️ Parando geração...</p>"
    print("\n⏹️ Interrupção solicitada pelo usuário")

start_generation_btn.on_click(start_generation_callback)
stop_generation_btn.on_click(stop_generation_callback)

# Layout de geração
generation_ui = widgets.VBox([
    widgets.HTML("<h3>🚀 Geração do Dataset</h3>"),
    widgets.HBox([start_generation_btn, stop_generation_btn]),
    progress_bar,
    status_text,
    generation_output
])

display(generation_ui)

## 📋 Formatação para YOLO

In [ ]:
# Interface de formatação YOLO
task_type_widget = widgets.Dropdown(
    options=[('Detecção', 'detection'), ('Segmentação', 'segmentation')],
    value='detection',
    description='Tarefa YOLO:'
)

train_split_widget = widgets.FloatSlider(
    value=0.7,
    min=0.5,
    max=0.8,
    step=0.05,
    description='Split Treino:'
)

val_split_widget = widgets.FloatSlider(
    value=0.2,
    min=0.1,
    max=0.3,
    step=0.05,
    description='Split Validação:'
)

test_split_widget = widgets.FloatSlider(
    value=0.1,
    min=0.05,
    max=0.2,
    step=0.05,
    description='Split Teste:'
)

generate_annotations_widget = widgets.Checkbox(
    value=True,
    description='Gerar Anotações Automáticas'
)

format_yolo_btn = widgets.Button(
    description='📋 Formatar para YOLO',
    button_style='info',
    layout=widgets.Layout(width='200px')
)

formatting_output = widgets.Output()

def format_yolo_callback(btn):
    global generated_dataset_path
    
    if not generated_dataset_path:
        with formatting_output:
            print("❌ Gere um dataset primeiro!")
        return
    
    with formatting_output:
        clear_output()
        
        try:
            print("📋 Formatando dataset para YOLO...")
            print(f"Tipo de tarefa: {task_type_widget.value}")
            print(f"Splits: {train_split_widget.value:.1f} / {val_split_widget.value:.1f} / {test_split_widget.value:.1f}")
            
            # Verificar se splits somam 1.0
            total_split = train_split_widget.value + val_split_widget.value + test_split_widget.value
            if abs(total_split - 1.0) > 0.01:
                print(f"⚠️ Ajustando splits (soma era {total_split:.2f})")
            
            formatter = YOLOFormatter()
            
            yolo_dataset_path = formatter.format_dataset(
                dataset_dir=generated_dataset_path,
                output_dir="/content/yolo_dataset",
                task_type=task_type_widget.value,
                train_split=train_split_widget.value,
                val_split=val_split_widget.value,
                test_split=test_split_widget.value,
                generate_annotations=generate_annotations_widget.value
            )
            
            # Validar dataset
            print("\n🔍 Validando dataset YOLO...")
            validation_results = formatter.validate_dataset(yolo_dataset_path)
            
            if validation_results['valid']:
                print("✅ Dataset YOLO válido!")
                print(f"📊 Splits criados: {validation_results['splits']}")
                print(f"📂 Dataset salvo em: {yolo_dataset_path}")
                
                # Mostrar arquivo de configuração
                config_file = Path(yolo_dataset_path) / "data.yaml"
                if config_file.exists():
                    print("\n📄 Configuração YOLO:")
                    with open(config_file, 'r') as f:
                        config_content = f.read()
                    print(config_content[:500] + "..." if len(config_content) > 500 else config_content)
                
            else:
                print(f"❌ Erro na validação: {validation_results.get('error', 'Desconhecido')}")
                
        except Exception as e:
            print(f"❌ Erro na formatação: {e}")

format_yolo_btn.on_click(format_yolo_callback)

# Layout formatação
formatting_ui = widgets.VBox([
    widgets.HTML("<h3>📋 Formatação YOLO</h3>"),
    task_type_widget,
    widgets.HTML("<h4>📊 Divisão do Dataset</h4>"),
    train_split_widget,
    val_split_widget,
    test_split_widget,
    generate_annotations_widget,
    format_yolo_btn,
    formatting_output
])

display(formatting_ui)

## 📊 Análise do Dataset Gerado

In [ ]:
def analyze_generated_dataset(dataset_path):
    """Analisa estatísticas do dataset gerado"""
    
    if not dataset_path or not Path(dataset_path).exists():
        print("❌ Dataset não encontrado")
        return
    
    dataset_dir = Path(dataset_path)
    
    # Carregar metadados
    metadata_file = dataset_dir / "dataset_metadata.json"
    if metadata_file.exists():
        import json
        with open(metadata_file, 'r') as f:
            metadata = json.load(f)
            
        print("📊 ANÁLISE DO DATASET GERADO")
        print("=" * 50)
        
        dataset_info = metadata.get('dataset_info', {})
        images = metadata.get('images', [])
        
        print(f"📈 Total de imagens: {dataset_info.get('total_images', 0)}")
        print(f"📅 Data de geração: {dataset_info.get('generation_date', 'N/A')}")
        
        # Estatísticas de geração
        stats = dataset_info.get('statistics', {})
        if stats:
            print(f"🎯 Taxa de sucesso: {stats.get('successful_generations', 0)}/{stats.get('total_attempts', 0)}")
            print(f"📊 Qualidade média: {stats.get('avg_quality_score', 0):.3f}")
            print(f"❌ Falhas de qualidade: {stats.get('quality_failures', 0)}")
            print(f"⚠️ Falhas técnicas: {stats.get('technical_failures', 0)}")
        
        # Análise de distribuições
        if images:
            biomes = {}
            seasons = {}
            qualities = {}
            coverages = []
            quality_scores = []
            
            for img in images:
                config = img.get('pasture_config', {})
                
                biome = config.get('biome')
                if biome:
                    biomes[biome] = biomes.get(biome, 0) + 1
                
                season = config.get('season')
                if season:
                    seasons[season] = seasons.get(season, 0) + 1
                
                quality = config.get('quality')
                if quality:
                    qualities[quality] = qualities.get(quality, 0) + 1
                
                coverage = config.get('grass_coverage')
                if coverage:
                    coverages.append(coverage)
                
                img_quality = img.get('quality_metrics', {}).get('overall_score')
                if img_quality:
                    quality_scores.append(img_quality)
            
            print("\n🌿 Distribuição por Bioma:")
            for biome, count in biomes.items():
                pct = count / len(images) * 100
                print(f"   {biome}: {count} ({pct:.1f}%)")
            
            print("\n🌦️ Distribuição Sazonal:")
            for season, count in seasons.items():
                pct = count / len(images) * 100
                print(f"   {season}: {count} ({pct:.1f}%)")
            
            print("\n🎯 Distribuição de Qualidade:")
            for quality, count in qualities.items():
                pct = count / len(images) * 100
                print(f"   {quality}: {count} ({pct:.1f}%)")
            
            if coverages:
                print(f"\n📊 Cobertura de Gramíneas:")
                print(f"   Média: {np.mean(coverages):.1f}%")
                print(f"   Min: {np.min(coverages)}%")
                print(f"   Max: {np.max(coverages)}%")
            
            if quality_scores:
                print(f"\n⭐ Scores de Qualidade:")
                print(f"   Média: {np.mean(quality_scores):.3f}")
                print(f"   Desvio: {np.std(quality_scores):.3f}")
                print(f"   Min: {np.min(quality_scores):.3f}")
                print(f"   Max: {np.max(quality_scores):.3f}")
                
                # Distribuição de qualidade
                excellent = sum(1 for s in quality_scores if s > 0.8)
                good = sum(1 for s in quality_scores if 0.6 <= s <= 0.8)
                moderate = sum(1 for s in quality_scores if 0.4 <= s < 0.6)
                poor = sum(1 for s in quality_scores if s < 0.4)
                
                print(f"\n📈 Distribuição de Qualidade:")
                print(f"   Excelente (>0.8): {excellent}")
                print(f"   Bom (0.6-0.8): {good}")
                print(f"   Moderado (0.4-0.6): {moderate}")
                print(f"   Ruim (<0.4): {poor}")
            
            # Visualização
            if coverages and quality_scores:
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
                
                # Histograma de cobertura
                ax1.hist(coverages, bins=20, alpha=0.7, color='green')
                ax1.set_xlabel('Cobertura de Gramíneas (%)')
                ax1.set_ylabel('Frequência')
                ax1.set_title('Distribuição de Cobertura')
                ax1.grid(True, alpha=0.3)
                
                # Histograma de qualidade
                ax2.hist(quality_scores, bins=20, alpha=0.7, color='blue')
                ax2.set_xlabel('Score de Qualidade')
                ax2.set_ylabel('Frequência')
                ax2.set_title('Distribuição de Qualidade')
                ax2.grid(True, alpha=0.3)
                
                plt.tight_layout()
                plt.show()
        
    else:
        print("❌ Arquivo de metadados não encontrado")

# Interface de análise
analyze_btn = widgets.Button(
    description='📊 Analisar Dataset',
    button_style='info'
)

analysis_output = widgets.Output()

def analyze_callback(btn):
    with analysis_output:
        clear_output()
        analyze_generated_dataset(generated_dataset_path)

analyze_btn.on_click(analyze_callback)

display(widgets.VBox([
    widgets.HTML("<h3>📊 Análise do Dataset</h3>"),
    analyze_btn,
    analysis_output
]))

## 📋 Resumo e Próximos Passos

### ✅ Funcionalidades implementadas:

1. **Configuração Flexível**: Ajuste de parâmetros via interface
2. **Geração Automática**: Pipeline completo com controle de qualidade
3. **Formatação YOLO**: Conversão automática com anotações
4. **Análise Estatística**: Visualização de distribuições e qualidade

### 🎯 Próximos notebooks:

1. **03_Quality_Control.ipynb**: Análise detalhada de qualidade
2. **04_YOLO_Training.ipynb**: Treinamento de modelos
3. **05_Evaluation_Benchmark.ipynb**: Avaliação e benchmarks

### 💡 Dicas importantes:

- **Memória**: Monitore uso de GPU, reinicie runtime se necessário
- **Qualidade**: Threshold 0.7+ recomendado para datasets finais
- **Distribuições**: Balance entre biomas conforme aplicação
- **ControlNet**: Melhora qualidade mas aumenta tempo de geração

### ⚠️ Troubleshooting:

- **Erro de CUDA**: Reinicie runtime com GPU
- **Falta de espaço**: Limpe outputs antigos
- **Geração lenta**: Reduza resolução ou desative ControlNet
- **Baixa qualidade**: Aumente threshold ou ajuste prompts